# BGP Shenanigans

## Connecting to a Device

Let's define some variables:

In [1]:
HOST = '172.26.170.252'
PORT = 830
USER = 'cisco'
PASS = 'C1sdevops'

Now let's establish a NETCONF session to that box using ncclient:

In [134]:
from ncclient import manager

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [8]:
print(m.server_capabilities)

['urn:ietf:params:xml:ns:yang:smiv2:CISCO-IPSLA-ECHO-MIB?module=CISCO-IPSLA-ECHO-MIB&revision=2007-08-16', 'urn:cisco:params:xml:ns:yang:cisco-qos-common?module=cisco-qos-common&revision=2015-05-09', 'http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z?module=SNMPv2-MIB&revision=2002-10-16', 'http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z?module=SNMP-NOTIFICATION-MIB&revision=2002-10-14', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-CBP-TARGET-TC-MIB?module=CISCO-CBP-TARGET-TC-MIB&revision=2006-03-24', 'urn:ietf:params:netconf:capability:notification:1.0', 'http://tail-f.com/ns/webui?module=tailf-webui&revision=2013-03-07', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-PRODUCTS-MIB?module=CISCO-PRODUCTS-MIB&revision=2014-11-06', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-UBE-MIB?module=CISCO-UBE-MIB&revision=2010-11-29', 'urn:ietf:params:netconf:capability:rollback-on-error:1.0', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-AAA-SESSION-MIB?module=CISCO-AAA-SESSION-MIB&revision=2006-03-2

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [14]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=report-all
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0


And now let's look at the capabilities that are related to model support:

In [36]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

CISCO-IPSLA-ECHO-MIB
  revision = 2007-08-16
cisco-qos-common
  revision = 2015-05-09
SNMPv2-MIB
  revision = 2002-10-16
SNMP-NOTIFICATION-MIB
  revision = 2002-10-14
CISCO-CBP-TARGET-TC-MIB
  revision = 2006-03-24
tailf-webui
  revision = 2013-03-07
CISCO-PRODUCTS-MIB
  revision = 2014-11-06
CISCO-UBE-MIB
  revision = 2010-11-29
CISCO-AAA-SESSION-MIB
  revision = 2006-03-21
tailf-common-monitoring
  revision = 2013-06-14
CISCO-RF-MIB
  revision = 2005-09-01
ENTITY-MIB
  revision = 2005-08-10
IP-FORWARD-MIB
  revision = 1996-09-19
cisco-qos-action-qlimit-cfg
  revision = 2015-05-09
  features = qos-grp-based-queuing-support,mpls-exp-based-queuing-support,disc-class-based-queuing-support
CISCO-BULK-FILE-MIB
  revision = 2002-06-10
iana-crypt-hash
  revision = 2014-04-04
  features = crypt-hash-sha-512,crypt-hash-sha-256,crypt-hash-md5
ENTITY-STATE-TC-MIB
  revision = 2005-11-22
EtherLike-MIB
  revision = 2003-09-19
CISCO-EIGRP-MIB
  revision = 2004-11-16
cisco-table-map
  revision = 201

## Enough!! What About Config?

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request

### But let's define some utilities first!

In [42]:
from lxml import etree

def pretty_print(nc_retval):
        print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))

### Getting Config

Let's look at some simple requests...

In [47]:
c = m.get_config(source='running')
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <device-model-version>
      <major>2</major>
      <minor>1</minor>
      <bug-fix>0</bug-fix>
    </device-model-version>
    <service>
      <timestamps>
        <debug>
          <datetime>
            <msec/>
          </datetime>
        </debug>
        <log>
          <datetime>
            <msec/>
          </datetime>
        </log>
      </timestamps>
    </service>
    <platform>
      <punt-keepalive>
        <disable-kernel-core>false</disable-kernel-core>
      </punt-keepalive>
    </platform>
    <enable>
      <password>
        <secret>C1sdevops</secret>
      </password>
    </enable>
    <username>
      <name>admin</name>
      <password>
        <encryption>0</encryption>
        <password>C1sdevops</password>
      </password>
    </username>
    <username>
      <name>cisco</name>
      <password>
    

Now let's add in a simple filter:

In [46]:
filter = '''
<netconf-yang/>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <netconf-yang xmlns="http://cisco.com/yang/cisco-self-mgmt">
    <cisco-ia xmlns="http://cisco.com/yang/cisco-ia">
      <auto-sync>without-defaults</auto-sync>
      <init-sync>without-defaults</init-sync>
      <intelligent-sync>false</intelligent-sync>
      <snmp-trap-control>
        <global-forwarding>true</global-forwarding>
        <trap-list>
          <trap-oid>1.3.6.1.4.1.9.9.41.2.0.1</trap-oid>
          <forward>true</forward>
        </trap-list>
        <trap-list>
          <trap-oid>1.3.6.1.6.3.1.1.5.3</trap-oid>
          <forward>true</forward>
        </trap-list>
        <trap-list>
          <trap-oid>1.3.6.1.6.3.1.1.5.4</trap-oid>
          <forward>true</forward>
        </trap-list>
      </snmp-trap-control>
      <message-diag-level>0</message-diag-level>
      <max-diag-messages-saved>30</max-diag-messages-saved>
      <post-sync-acl-process>true</post

In [48]:
filter = '''
<native>
  <ip>
    <route/>
  </ip>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <ip>
      <route>
        <ip-route-interface-forwarding-list>
          <prefix>0.0.0.0</prefix>
          <mask>0.0.0.0</mask>
          <fwd-list>
            <fwd>172.26.170.1</fwd>
          </fwd-list>
        </ip-route-interface-forwarding-list>
        <vrf>
          <name>Mgmt-vrf</name>
          <ip-route-interface-forwarding-list>
            <prefix>0.0.0.0</prefix>
            <mask>0.0.0.0</mask>
            <fwd-list>
              <fwd>172.26.170.1</fwd>
            </fwd-list>
          </ip-route-interface-forwarding-list>
        </vrf>
      </route>
    </ip>
  </native>
</data>



Now let's look at the BGP native model:

In [135]:
filter = '''
<native>
  <router>
    <bgp/>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <router>
      <bgp>
        <id>1</id>
        <bgp>
          <graceful-restart/>
          <log-neighbor-changes/>
        </bgp>
        <neighbor>
          <id>192.168.1.1</id>
          <remote-as>666</remote-as>
          <description>modified at 1466006302.11</description>
          <ebgp-multihop>
            <max-hop>255</max-hop>
          </ebgp-multihop>
        </neighbor>
      </bgp>
    </router>
  </native>
</data>



And can we look at a specific neighbor only? Say the one with id (address) '192.168.1.1'?

In [138]:
filter = '''
<native>
  <router>
    <bgp>
      <neighbor>
        <id>192.168.1.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <router>
      <bgp>
        <neighbor>
          <id>192.168.1.1</id>
          <remote-as>666</remote-as>
          <description>modified at 1466008798.62</description>
          <ebgp-multihop>
            <max-hop>255</max-hop>
          </ebgp-multihop>
        </neighbor>
      </bgp>
    </router>
  </native>
</data>



Ok, so, yes we can get a specific neighbor. Now, can we modify something in a neighbor? Let's keep it simple and modify the description:

In [137]:
import time
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <router>
      <bgp>
        <id>1</id>
        <neighbor>
          <id>192.168.1.1</id>
          <description nc:operation="merge">modified at {}</description>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''.format(time.time())
try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Other Stuff

Get interface data from native model:

In [142]:
filter = '''
<native>
  <interface>
    <GigabitEthernet>
      <name>1/0/1</name>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <interface>
      <GigabitEthernet>
        <name>1/0/1</name>
        <switchport-conf>
          <switchport>false</switchport>
        </switchport-conf>
        <description>boom</description>
        <ip>
          <address>
            <primary>
              <address>1.1.1.1</address>
              <mask>255.255.254.0</mask>
            </primary>
          </address>
        </ip>
      </GigabitEthernet>
    </interface>
  </native>
</data>



Get aaa stuff:

In [141]:
filter = '''
<native>
  <aaa/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <aaa>
      <new-model/>
      <session-id>common</session-id>
    </aaa>
  </native>
</data>

